#  Car project - part 2

# Roi sharabi 318306859 Raz ben ami 208853259

# github url 'https://github.com/raza783/cars-project-part-2'

In [36]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet


In [37]:
df = pd.read_csv('https://raw.githubusercontent.com/raza783/cars-project-part-2/main/dataset.csv')

### data understanding

In [38]:
df.head()

,manufactor,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
0,יונדאי,2015,i35,2,אוטומטית,1600,בנזין,פרטית,פרטית,רעננה - כפר סבא,רעננה,51000.0,2.0,11/07/2023,11/07/2023,['רכב שמור בקנאות\nמוכרת עקב קבלת רכב חברה'],כחול כהה מטאלי,144000,NaN,NaN
1,ניסאן,2018,ניסאן מיקרה,1,אוטומטית,1200,בנזין,פרטית,פרטית,מושבים בשרון,אבן יהודה,49000.0,0.0,06/04/2022,22/05/2022,['שמורה כל התוספות'],כחול בהיר,69000,NaN,NaN
2,סוזוקי,2010,סוזוקי סוויפט,1,אוטומטית,1450,בנזין,NaN,NaN,רמת,רמת,22500.0,1.0,29/10/2022,29/10/2022,['רכב במצב מתוחזק ברמה גבוהה טסט עד אפריל 2023'],None,145000,NaN,NaN
3,טויוטה,2016,אוריס,1,טיפטרוניק,1600,בנזין,פרטית,פרטית,נס ציונה - רחובות,רחובות,63000.0,5.0,16/05/2024,16/05/2024,['אוטו במצב חדש!! שמור בקנאות!! נהג יחיד מטופל...,אפור מטאלי,27300,NaN,NaN
4,קיה,2012,פיקנטו,1,אוטומטית,1248,בנזין,NaN,NaN,"ראשל""צ והסביבה",ראשון לציון,37000.0,1.0,13/06/2022,13/06/2022,['שמור'],NaN,70000,NaN,4.0


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   manufactor       1500 non-null   object 
 1   Year             1500 non-null   int64  
 2   model            1500 non-null   object 
 3   Hand             1500 non-null   int64  
 4   Gear             1499 non-null   object 
 5   capacity_Engine  1474 non-null   object 
 6   Engine_type      1495 non-null   object 
 7   Prev_ownership   774 non-null    object 
 8   Curr_ownership   774 non-null    object 
 9   Area             1361 non-null   object 
 10  City             1500 non-null   object 
 11  Price            1500 non-null   float64
 12  Pic_num          1476 non-null   float64
 13  Cre_date         1500 non-null   object 
 14  Repub_date       1500 non-null   object 
 15  Description      1500 non-null   object 
 16  Color            787 non-null    object 
 17  Km            

### We can see from the data what is missing in each column, and accordingly, we will know which columns we need to fill.

In [40]:
df.describe()

,Year,Hand,Price,Pic_num,Supply_score
count,1500.000000,1500.000000,1500.000000,1476.000000,439.000000
mean,2014.346000,2.349333,51085.086000,2.836721,1581.011390
std,3.815406,1.229217,21933.308735,3.499312,2651.323149
min,1983.000000,1.000000,18200.000000,0.000000,0.000000
25%,2012.000000,1.000000,32000.000000,1.000000,20.000000
50%,2015.000000,2.000000,48000.000000,1.000000,474.000000
75%,2017.000000,3.000000,68000.000000,4.000000,2402.000000
max,2023.000000,10.000000,99960.000000,40.000000,16508.000000


### part 1 - prepare data function : 
#### 1. remove duplicates
#### 2. fill missing values specific to any columm
#### 3. adapt categorial features to categorial
#### 4. adapt numeric features to numeric including convert values types
#### 5. return processed data for next step





In [41]:
def prepare_data(df):
    # Create a copy of the original DataFrame
    df = df.copy()

    # Remove duplicates
    df = df.drop_duplicates()

    # Handle missing values in categorical columns by sampling from existing values,we did it because we want to take random sample from the column distribution
    
    prev_ownership_values = df['Prev_ownership'].dropna().values
    df.loc[pd.isnull(df['Prev_ownership']), 'Prev_ownership'] = np.random.choice(prev_ownership_values, pd.isnull(df['Prev_ownership']).sum())
    
    curr_ownership_values = df['Curr_ownership'].dropna().values
    df.loc[pd.isnull(df['Curr_ownership']), 'Curr_ownership'] = np.random.choice(curr_ownership_values, pd.isnull(df['Curr_ownership']).sum())
    
    color_values = df['Color'].dropna().values
    df.loc[pd.isnull(df['Color']), 'Color'] = np.random.choice(color_values, pd.isnull(df['Color']).sum())

    # Convert 'Test' column to the number of days since the test date
    df.loc[:, 'Test'] = pd.to_datetime(df['Test'], errors='coerce')
    df.loc[:, 'Test'] = (pd.Timestamp.now() - df['Test']).dt.days
    Test_values = df['Test'].dropna().values
    df.loc[pd.isnull(df['Test']), 'Test'] = np.random.choice(Test_values, pd.isnull(df['Test']).sum())
   
    # Handle missing values in other columns
    df.loc[:, 'Gear'] = df['Gear'].fillna(df['Gear'].mode()[0])
    
    # Remove commas and convert numeric values
    df.loc[:, 'capacity_Engine'] = df['capacity_Engine'].replace(',', '', regex=True)
    df.loc[:, 'capacity_Engine'] = pd.to_numeric(df['capacity_Engine'], errors='coerce')  # Handle non-numeric values
   
    # Convert invalid values in the 'Km' column
    df.loc[:, 'Km'] = df['Km'].replace(',', '', regex=True)
    df.loc[:, 'Km'] = pd.to_numeric(df['Km'], errors='coerce')
    
    # handle missing values using groupby and fill with median or mean
    
    df.loc[:, 'capacity_Engine'] = df.groupby(['manufactor', 'model'])['capacity_Engine'].transform(lambda x: x.fillna(x.median()))
    df.loc[:, 'Engine_type'] = df.groupby('model')['Engine_type'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else ''))
    df.loc[:, 'Area'] = df['Area'].fillna(df['Area'].mode()[0])
    df.loc[:, 'Pic_num'] = df['Pic_num'].fillna(df['Pic_num'].median())
    df.loc[:, 'Km'] = df.groupby('Year')['Km'].transform(lambda x: x.fillna(x.median()))
    df.loc[:, 'Supply_score'] = df.groupby('manufactor')['Supply_score'].transform(lambda x: x.fillna(x.median()))

    # Convert categorical columns to categories
    df.loc[:, 'manufactor'] = df['manufactor'].astype('category')
    df.loc[:, 'model'] = df['model'].astype('category')
    df.loc[:, 'Gear'] = df['Gear'].astype('category')
    df.loc[:, 'Engine_type'] = df['Engine_type'].astype('category')
    df.loc[:, 'Area'] = df['Area'].astype('category')
    df.loc[:, 'City'] = df['City'].astype('category')
    df.loc[:, 'Color'] = df['Color'].astype('category')
    df.loc[:, 'Prev_ownership'] = df['Prev_ownership'].astype('category')
    df.loc[:, 'Curr_ownership'] = df['Curr_ownership'].astype('category')

    # Remove outliers by calculating values outside the interquartile range (IQR)
    #in this part we understand that we want to prevent outliers values and we check for function that deal with that
    #we understand that this function can be dangerous because overffiting and we will solve it in the fold cross validation check
    numeric_columns = ['Year', 'Hand', 'capacity_Engine', 'Km', 'Test', 'Supply_score', 'Pic_num', 'Price']
    Q1 = df[numeric_columns].quantile(0.15)
    Q3 = df[numeric_columns].quantile(0.85)
    IQR = Q3 - Q1
    df = df[~((df[numeric_columns] < (Q1 - 1.5 * IQR)) | (df[numeric_columns] > (Q3 + 1.5 * IQR))).any(axis=1)]

    return df

# Prepare the data
df_processed = prepare_data(df)


### part 2  : 
#### 1. definition of the X,Y parameters
#### 2. transformators definition using simple imputer,one hot encoder and piplines
#### 3. dealing with infinity,Nan values of  the train set
#### 4. define elastic net model
#### 5. build parameters search to get the most effective parameters for the smallest RMSE
#### 6. return the best model with the parameters of alpha and l1_ratio
#### 7. provide the RMSE score 

In [42]:
# Define relevant columns
features = ['manufactor', 'Year', 'model', 'Hand', 'Gear', 'capacity_Engine', 'Engine_type', 
            'Prev_ownership', 'Curr_ownership', 'Area', 'City', 'Km', 'Test', 
            'Supply_score', 'Pic_num', 'Color']
target = 'Price'

X = df_processed[features]
y = df_processed[target]

# Define numeric and categorical columns
numeric_features = ['Year', 'Hand', 'capacity_Engine', 'Km', 'Test', 'Supply_score', 'Pic_num']
categorical_features = ['manufactor', 'model', 'Gear', 'Engine_type', 'Area', 'City', 'Color', 'Prev_ownership', 'Curr_ownership']

# Define transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder='passthrough')

X = preprocessor.fit_transform(X)

In [43]:
# Convert to a dense array to handle NaN and infinite values
X = X.toarray()

# Convert to DataFrame to handle NaN and infinite values
X = pd.DataFrame(X).astype(float)

# Fill NaN values with the median of the column
X.fillna(X.median(), inplace=True)

# Replace infinite values with the maximum value in the column
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(X.median(), inplace=True)

In [44]:
# Define the model
model = ElasticNet(random_state=42)

# Define the parameter grid for searching
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0]
}

# Define GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=10, n_jobs=-1)

# Perform the grid search
grid_search.fit(X, y)

# Display the best parameters found
print("Best parameters found: ", grid_search.best_params_)
print("Best RMSE: ", -grid_search.best_score_)

# Train the model with the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X, y)

Best parameters found:  {'alpha': 0.01, 'l1_ratio': 0.9}
Best RMSE:  10706.276072447235


ElasticNet(alpha=0.01, l1_ratio=0.9, random_state=42)

In [45]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Perform 10-fold cross-validation
cv_scores = cross_val_score(best_model, X, y, cv=10, scoring='neg_root_mean_squared_error')

# Print all cross-validation RMSE scores
print("Cross-validation RMSE scores: ", -cv_scores)

# Calculate the average RMSE
mean_rmse = -np.mean(cv_scores)
std_rmse = np.std(cv_scores)

print("Mean RMSE: ", mean_rmse)
print("Standard Deviation RMSE: ", std_rmse)

# Calculate additional performance metrics
predictions = best_model.predict(X)
mae = mean_absolute_error(y, predictions)
r2 = r2_score(y, predictions)

print("Mean Absolute Error: ", mae)
print("R^2 Score: ", r2)

# Explanation of performance metrics:
# RMSE (Root Mean Squared Error): This metric measures the average deviation of the model's predictions from the actual values. The lower the RMSE, the more accurate the model.
# The mean RMSE was calculated by averaging the results from cross-validation.
# The standard deviation of RMSE indicates the variability of the scores obtained from cross-validation.

# MAE (Mean Absolute Error): This metric measures the average absolute errors of the model. Like RMSE, the lower the MAE, the more accurate the model.

# R^2 (R-squared): This metric explains the proportion of the variance in the dependent variable that is predictable from the independent variables. R^2 values range from 0 to 1, with 1 indicating a model that perfectly explains the variance in the data.


Cross-validation RMSE scores:  [10559.45689743  8989.6948417  10122.85806493  9134.76151798
 12789.53493775 10410.69904042 10253.70760698 13011.60759606
 12097.27756252  9693.1626587 ]
Mean RMSE:  10706.276072447235
Standard Deviation RMSE:  1366.9758748507636
Mean Absolute Error:  5326.50064167168
R^2 Score:  0.900870673910309


In [46]:
# Get the feature names from the preprocessor
numeric_feature_names = numeric_features
categorical_feature_names = preprocessor.transformers_[1][1]['onehot'].get_feature_names(categorical_features)
feature_names = np.concatenate([numeric_feature_names, categorical_feature_names])

# Create a DataFrame for feature importances
feature_importances = pd.DataFrame({
    'Feature': feature_names,
    'Importance': best_model.coef_
})

# Sort the DataFrame by absolute importance
feature_importances['Absolute Importance'] = feature_importances['Importance'].abs()
feature_importances = feature_importances.sort_values(by='Absolute Importance', ascending=False)

# Get top 5 features
top_5_features = feature_importances.head(5)

# Print the top 5 features
print("\nTop 5 features:")
for index, row in top_5_features.iterrows():
    effect = "Positive" if row['Importance'] > 0 else "Negative"
    print(f"{row['Feature']}: {row['Importance']:.4f} ({effect})")




Top 5 features:
model_I-MIEV: 29076.1431 (Positive)
model_RCZ: 22911.4243 (Positive)
model_ספייס סטאר: -22579.7076 (Negative)
model_ S-Class: 20140.0632 (Positive)
Area_רחובות: 18884.9903 (Positive)
